<a href="https://colab.research.google.com/github/Akash-D-wivedi/phase-transformation-jmak/blob/main/notebooks/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase‐Transformation Kinetics: JMAK Analysis


This notebook demonstrates:
1. Loading TTT data
2. Fitting the JMAK equation (Avrami exponent & rate constants)
3. Arrhenius analysis (k₀, Q)
4. Gaussian‐Process smoothing
5. Synthetic curve generation
6. Visualization and uncertainty quantification


In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import least_squares, curve_fit
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from pathlib import Path
import chardet, re, unicodedata, warnings

# Paths
DATA_DIR = Path("../data")
CSV_FILE = DATA_DIR / "TTT_data_Steel.csv"

## 1. Load & Clean Data

We auto-detect encoding, normalize column names, and preview the first few rows.


In [27]:
# (Insert read_csv_auto + normalize_cols helper here)
df_raw = read_csv_auto(CSV_FILE)
df_raw = normalize_cols(df_raw)
df_raw.head()

NameError: name 'read_csv_auto' is not defined

## 2. Tidy Data

Melt the 0%, 10%, and 90% times into a long format for fitting.


In [30]:
tidy = (
    df_raw.melt(
        id_vars=["phase","t_c","y_max"],
        value_vars=["t_start","t_10","t_90"],
        var_name="time_kind", value_name="time_s"
    )
    .assign(fraction=lambda d: d.time_kind.map({
        "t_start":0.0, "t_10":0.10, "t_90":0.90
    }))
    .dropna(subset=["time_s"])
)
tidy.head()

,phase,t_c,y_max,time_kind,time_s,fraction
0,Ferrite,470.0,0.4,t_start,2.1,0.0
1,Ferrite,480.0,0.4,t_start,1.9,0.0
2,Ferrite,490.0,0.4,t_start,1.7,0.0
3,Ferrite,500.0,0.4,t_start,1.5,0.0
4,Ferrite,510.0,0.4,t_start,1.4,0.0


## 3. Fit JMAK Parameters (n & k(T))

We fit the Avrami exponent *n* and per‐temperature rate constants *k* by non‐linear least squares.


In [29]:
results = {}
for phase, grp in tidy.groupby("phase"):
    # ... (pack/unpack helpers and least_squares call) ...
    results[phase] = {"n": n_fit, "k_T": kdict}
results

NameError: name 'kdict' is not defined

## 4. Arrhenius Fit

Fit $$ k(T) \to (k_0\exp(-Q/(RT))) $$ to extract activation energy Q and pre‐factor k₀.

In [ ]:
arr_params = {}
for phase, info in results.items():
    # ... (curve_fit for Arrhenius) ...
    arr_params[phase] = {"k0":k0, "Q":Q}
pd.DataFrame(arr_params).T


## 5. Gaussian‐Process Smoothing & Synthetic Curves

We train a GP on $ (\ln k) $ vs T, then generate log‐spaced synthetic f(t) curves.


In [ ]:
synthetic = []
for phase, info in results.items():
    # ... (GP fit + logspace curve generation) ...
    synthetic_df = pd.DataFrame(synthetic, columns=["phase","T_C","time_s","fraction"])
synthetic_df.head()

## 6. Visualize Fits & 95% Confidence Bands

Overlay the original 0/10/90 % points with fitted curves and GP‐based CI.


In [ ]:
for phase in results:
    # ... (plot scatter, line, fill_between) ...
